<a href="https://colab.research.google.com/github/marconebhering/matriz-rfm/blob/main/RFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_excel("https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx")

In [ ]:
df = pd.read_excel("https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx")

## Limpeza

In [ ]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
print('O dataset possui {} registros e {} colunas'.format(df.shape[0], df.shape[1]))

O dataset possui 541909 registros e 8 colunas


### Valores Ausentes

In [ ]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Vemos valores ausentes nas colunas `Description` e `CustomerID`, neste caso removeremos estes registros.

In [ ]:
# Dropando valores ausentes e checando o tamanho do dataset
df.dropna(inplace=True)
df.shape

(406829, 8)

### Registros Duplicados

In [ ]:
print('Temos ainda {} registros duplicados.'.format(df.duplicated().sum()))

Temos ainda 5225 registros duplicados.


In [ ]:
# Dropando registros duplicados e checando o tamanho do dataset
df.drop_duplicates(inplace=True)
df.shape

(401604, 8)

Com isso finalizamos a etapa de limpeza do conjunto de dados.

## Tratamento

In [ ]:
# Última data de compra
data_corte = df['InvoiceDate'].max()

In [ ]:
df['amount'] = df['Quantity']*df['UnitPrice']

In [ ]:
df_rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (data_corte - x.max()).days,
    'CustomerID': 'count',
    'amount': 'sum'})

In [ ]:
df_rfm.rename(columns={'InvoiceDate': 'recencia',
                   'CustomerID': 'frequencia',
                   'amount': 'valor'}, inplace=True)

In [ ]:
df_rfm.head()

,recencia,frequencia,valor
CustomerID,,,
12346.0,325,2,0.00
12347.0,1,182,4310.00
12348.0,74,31,1797.24
12349.0,18,73,1757.55
12350.0,309,17,334.40


In [ ]:
df_rfm_clean = df_rfm[df_rfm['valor'] != 0]

## Transformações

Antes de importar os dados para a ferramenta de visualização, vamos realizar algumas etapas de transformações.

In [ ]:
# Criando os níveis de R, F e M - 5 níveis
levels_r =range(5,0,-1)
levels_f=range(1,6)
levels_m=range(1,6)

# Dividindo a lista de clientes em 5 quintis (dividindo nos quartis)
r_quint = pd.qcut(df_rfm_clean['recencia'],  q=5, labels = levels_r)
f_quint = pd.qcut(df_rfm_clean['frequencia'], q=5, labels = levels_f)
m_quint = pd.qcut(df_rfm_clean['valor'], q=5, labels = levels_m)

rfm = df_rfm_clean.assign(R=r_quint, F=f_quint, M=m_quint)

# Criando tabela atribuindo níveis RFM e pontuação RFM (soma dos níveis)
def add_rfm(x): return str(x['R']) + str(x['F']) + str(x['M'])

rfm['RFM_cluster'] = rfm.apply(add_rfm,axis=1 )
rfm['RFM_score'] = rfm[['R','F','M']].sum(axis=1)
rfm['FM_mean'] = rfm[['F','M']].mean(axis=1).round()
rfm = rfm.reset_index()

In [ ]:
rfm['R'] = rfm['R'].astype('int64')
rfm['FM_mean'] = rfm['FM_mean'].astype('int64')

def classify(df):
    if (df['FM_mean'] == 5) and (df['R'] == 1):
        return "Can't Lose"
    elif (df['FM_mean'] == 5) and ((df['R'] == 3) or (df['R'] == 4)):
        return 'Loyal Customer'
    elif (df['FM_mean'] == 5) and (df['R'] == 5):
        return 'Champion'
    elif (df['FM_mean'] == 4) and (df['R'] >= 3):
        return 'Loyal Customer'    
    elif (df['FM_mean'] == 3) and (df['R'] == 3):
        return 'Need Atention'    
    elif ((df['FM_mean'] == 3) or (df['FM_mean'] == 2))  and (df['R'] > 3):
        return 'Potential Loyalist' 
    elif ((df['FM_mean'] == 2) or (df['FM_mean'] == 1)) and (df['R'] == 1):
        return 'Lost'     
    elif (df['FM_mean'] == 2) and (df['R'] == 2):
        return 'Hibernating'     
    elif ((df['FM_mean'] == 2) or (df['R'] == 1)) and (df['R'] == 3):
        return 'About to Sleep'
    elif (df['FM_mean'] == 1) and (df['R'] == 2):
        return 'Lost'
    elif (df['FM_mean'] == 1) and (df['R'] == 4):
        return 'Promising'       
    elif (df['FM_mean'] == 1) and (df['R'] == 5):
        return 'New Customers'  
    else:
        return 'At Risk'
rfm['segmento'] = rfm.apply(classify, axis=1)

In [ ]:
rfm.head()

,CustomerID,recencia,frequencia,valor,R,F,M,RFM_cluster,RFM_score,FM_mean,segmento
0,12347.0,1,182,4310.00,5,5,5,5.05.05.0,15,5,Champion
1,12348.0,74,31,1797.24,2,3,4,2.03.04.0,9,4,At Risk
2,12349.0,18,73,1757.55,4,4,4,4.04.04.0,12,4,Loyal Customer
3,12350.0,309,17,334.40,1,2,2,1.02.02.0,5,2,Lost
4,12352.0,35,95,1545.41,3,4,4,3.04.04.0,11,4,Loyal Customer


Finalmente nosso conjunto de dados está pronto para ser importado para o Looker Studio, onde iremos construir nosso Dashboard.

In [ ]:
rfm.to_csv('rfm.csv', index=False)